# Sumar capital anterior bueno con diferencias calculadas

Este script recibe como insumos el informe anterior bueno y el csv de diferencias calculadas entre el anterior malo y el nuevo malo para calcular el informe nuevo bueno.

anteriorBueno + (nuevoMalo - anteriorMalo) = nuevoBueno

In [1]:
import pandas as pd
def tonumberNeg(text):
    text = text.replace('.', '')
    text = text.replace(',', '')
    return -int(text)

def tonumberPos(text):
    text = text.replace('.', '')
    text = text.replace(',', '')
    return int(text)
def leerInforme(nombre):
    df = pd.read_csv("csv/{}.csv".format(nombre), delimiter=",")
    # Dejar solo nombre y saldo
    saldos = df[df['Vencto.'] == 'Total'][['TP.1', 'Unnamed: 8', 'Créditos', 'Saldo']]
    # Dejar solo nombre y rut
    ruts = df[(~df['TP'].isna()) & (df['TP']!='VO')][['TP', 'Vencto.']]
    # Cambiar nombres de columnas a nombres buenos
    ruts.columns = ['Rut', 'Nombre']
    saldos.columns = ['Nombre', 'Debito','Credito', 'Saldo']
    # Especificar el tipo a la columna nombre
    ruts['Nombre'] = ruts['Nombre'].astype('string')
    saldos['Nombre'] = saldos['Nombre'].astype('string')
    # Transformar columna saldo a numerica
    saldos['Saldo'] = saldos['Saldo'].apply(tonumberNeg)
    saldos['Debito'] = saldos['Debito'].fillna('0').apply(tonumberPos)
    saldos['Credito'] = saldos['Credito'].fillna('0').apply(tonumberPos)
    # Sumar saldos de nombres repetidos y quitar nombres repetidos con diferentes ruts
    saldos = saldos.groupby('Nombre').sum().reset_index()
    ruts = ruts[~ruts.duplicated(subset="Nombre", keep='first')]
    print('len de saldos: {}, len de ruts: {} '.format(len(saldos), len(ruts)))
    # ordenar ruts
    ruts = ruts.sort_values(by=['Nombre'])
    #saldos = saldos.sort_values(by=['Nombre']) para probar pero el error era blumel
    # Join de tablas saldos y ruts para crear Capital.
    capital = saldos.reset_index(drop=True).join(ruts.reset_index(drop=True), rsuffix="_")
    print("Aseguarse que las filas fueron bien unidas si las unicas filas no iguales en la realidad corresponden:")
    print(capital[capital['Nombre']!=capital['Nombre_']])
    # Quitar fila de nombre extra
    capital = capital[['Nombre', 'Debito', 'Credito', 'Saldo', 'Rut']]

    #Quitar los que tengan saldo 0
    capital = capital[(capital['Saldo']>0) | (capital['Saldo']<0)]
    # Transformar a string
    capital['Nombre'] = capital['Nombre'].astype('string')
    # Transformar rut a string, quitar espacios vacios y transformar a minuscula.
    capital['Rut'] = capital['Rut'].astype('string').apply(str.strip).apply(str.lower)

    return capital

## Aqui empieza el script
Cada mes hay que cambiar Diferencias{new}{old}.csv y {old}Bueno

In [2]:
nuevomes = "Enero"
antmes = "Diciembre"
nuevo = nuevomes[:3]
viejo = antmes[:3]

In [3]:
df = pd.read_csv("diferencias/Diferencias{}{}.csv".format(nuevo,viejo))
dif = df[df['diff']!=0][["Nombre", "diff_debito", "diff_credito", "diff", "Rut_desp"]]

In [4]:
# anteriorBueno = leerInforme("2023enero")
anteriorBueno = pd.read_csv("{}Bueno.csv".format(antmes), index_col="Unnamed: 0")

In [5]:
dif.columns = ["Nombre", "Debito", "Credito", "Saldo", "Rut"]
print(dif.columns)
print(anteriorBueno.columns)

Index(['Nombre', 'Debito', 'Credito', 'Saldo', 'Rut'], dtype='object')
Index(['Rut', 'Debito', 'Credito', 'Saldo', 'Cuotas', 'Nombre'], dtype='object')


In [6]:
pd.set_option('display.max_rows', 10)
#
merged_df = pd.concat([anteriorBueno, dif]).groupby(['Rut']).sum().reset_index()

final_df = pd.merge(merged_df, pd.concat([anteriorBueno, dif])[["Rut", "Nombre"]], on='Rut')
final_df[final_df.duplicated(subset=["Rut", "Saldo"], keep=False)]

C:\Users\neira\AppData\Local\Temp\ipykernel_3440\1605799583.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  merged_df = pd.concat([anteriorBueno, dif]).groupby(['Rut']).sum().reset_index()


,Rut,Debito,Credito,Saldo,Cuotas,Nombre
0,10041916-5,0.0,178338.0,178338.0,168.0,ELEONOR MARINA HUECHUCURA CARO
1,10041916-5,0.0,178338.0,178338.0,168.0,ELEONOR MARINA HUECHUCURA CARO
2,10046759-3,0.0,741533.0,741533.0,604.0,JACQUELINE SOLANGE CLAVERIE GATICA
3,10046759-3,0.0,741533.0,741533.0,604.0,JACQUELINE SOLANGE CLAVERIE GATICA
4,10047568-5,0.0,775872.0,775872.0,663.0,CARMEN LUCY OBREQUE MORALES
...,...,...,...,...,...,...
1444,u1223501,0.0,93989.0,93989.0,88.0,CHRISTIAN FELBER
1445,wolf2017,0.0,44886.0,44886.0,42.0,WOLF HOLGER VAN DER MULBE
1446,wolf2017,0.0,44886.0,44886.0,42.0,WOLF HOLGER VAN DER MULBE
1447,x4879185,718176.0,1436352.0,718176.0,678.0,CORINA SCHAUB


In [7]:
final_df = final_df.drop_duplicates(subset=['Rut'], keep='first').reset_index(drop=True)
final_df.drop(final_df[final_df["Saldo"]==0].index, inplace=True)

In [8]:
final_df["Saldo"].sum()

172391815.0

In [17]:
# Guardar dataframe último bueno
final_df.to_csv("{}Bueno.csv".format(nuevomes))

In [18]:
capital = final_df
capital['Cuotas'] = capital['Saldo'].apply(lambda x: x//1000)

In [19]:
cred = {}
cred['1'] = int(capital[capital['Saldo']<=10000]['Credito'].sum())
cred['2'] = int(capital[(capital['Saldo']>10000)&(capital['Saldo']<=50000)]['Credito'] .sum())
cred['3'] = int(capital[(capital['Saldo']>50000)&(capital['Saldo']<=100000)]['Credito'].sum())
cred['4'] = int(capital[capital['Saldo']>100000]['Credito'].sum())
cred['total'] = int(capital['Credito'].sum())
assert cred['1']+cred['2']+cred['3']+cred['4'] == cred['total']
cred

{'1': 11706361,
 '2': 12552937,
 '3': 9459502,
 '4': 163673420,
 'total': 197392220}

In [20]:
deb = {}
deb['1'] = int(capital[capital['Saldo']<=10000]['Debito'].sum())
deb['2'] = int(capital[(capital['Saldo']>10000)&(capital['Saldo']<=50000)]['Debito'] .sum())
deb['3'] = int(capital[(capital['Saldo']>50000)&(capital['Saldo']<=100000)]['Debito'].sum())
deb['4'] = int(capital[capital['Saldo']>100000]['Debito'].sum())
deb['total'] = int(capital['Debito'].sum())
assert deb['1']+deb['2']+deb['3']+deb['4'] == deb['total']
deb

{'1': 9820975, '2': 7700814, '3': 2303696, '4': 11496584, 'total': 31322069}

In [21]:
sald = {}
sald['1'] = [int(capital[capital['Saldo']<=10000]['Saldo'].sum()), capital[capital['Saldo']<=10000]['Saldo'].count()]
sald['2'] = [int(capital[(capital['Saldo']>10000)&(capital['Saldo']<=50000)]['Saldo'] .sum()), capital[(capital['Saldo']>10000)&(capital['Saldo']<=50000)]['Saldo'] .count()]
sald['3'] = [int(capital[(capital['Saldo']>50000)&(capital['Saldo']<=100000)]['Saldo'].sum()), capital[(capital['Saldo']>50000)&(capital['Saldo']<=100000)]['Saldo'].count()]
sald['4'] = [int(capital[capital['Saldo']>100000]['Saldo'].sum()), capital[capital['Saldo']>100000]['Saldo'].count()]
sald['total'] = int(capital['Saldo'].sum())
assert sald['1'][0]+sald['2'][0]+sald['3'][0]+sald['4'][0] == sald['total']
sald

{'1': [1885386, 407],
 '2': [4852123, 187],
 '3': [7155806, 96],
 '4': [152176836, 233],
 'total': 166070151}

In [22]:
assert cred['total']-deb['total'] == sald['total']

In [27]:
import docx
from docx.shared import Pt
from docx.enum.section import WD_ORIENTATION
def crearDocumentoWord(capitales,ruta):
    MES = nuevomes.capitalize()
    ANIO = "2024"
    
    ruta = ruta+"/CapitalPagado{}.docx".format(path_capital)

    personasTramo1 = sald['1'][1]
    personasTramo2 = sald['2'][1]
    personasTramo3 = sald['3'][1]
    personasTramo4 = sald['4'][1]

    totalCreditos = int(cred['total'])
    totalDebitos = int(deb['total'])
    saldoAcreedor = int(sald['total'])
    totalSocios = sum([sald['1'][1], sald['2'][1], sald['3'][1], sald['4'][1]])

    

    informe = docx.Document()
    # Configuracion fuente, orientacion
    section = informe.sections[-1]
    new_width, new_height = section.page_height, section.page_width
    section.orientation = WD_ORIENTATION.LANDSCAPE
    section.page_width = new_width
    section.page_height = new_height
    style = informe.styles['Normal']
    font = style.font
    font.name = 'Calibri'
    font.size = Pt(14)
    # Escribir los parrafos
    informe.add_paragraph("\t\tCuenta\t\t\t\t\tCapital Pagado.")
    informe.save(ruta)
    informe.add_paragraph("\t\tMes\t\t\t\t\t\t" + MES + " " + ANIO)
    informe.save(ruta)

    datosTributarios = "\t\tDatos del Balance tributario:"
    datosTributarios += "\n\t\tTotal Creditos\t\t\t\t$" + '{:,}'.format(totalCreditos).replace(',', '.')
    datosTributarios += "\n\t\tTotal Debitos\t\t\t\t$  " + '{:,}'.format(totalDebitos).replace(',', '.')
    datosTributarios += "\n\t\tSaldo Acreedor\t\t\t\t$" + '{:,}'.format(saldoAcreedor).replace(',', '.')
    datosTributarios += "\n\t\t\t\t\t\t\t\t============"
    informe.add_paragraph(datosTributarios)
    informe.save(ruta)

    informe.add_paragraph("Clasificación aportes por socio")
    informe.save(ruta)
    informe.add_paragraph("Desde\t\t\tHasta\t\tN.º socios\t\tDébitos\t\tCréditos\t\tSaldo")
    informe.save(ruta)

    doc_tramo1 = str(personasTramo1) + "\t\t$" + '{:,}'.format(deb['1']).replace(',', '.')
    doc_tramo1 += "\t\t$" + '{:,}'.format(cred['1']).replace(',', '.')
    doc_tramo1 += "\t\t$" + '{:,}'.format(sald['1'][0]).replace(',', '.')
    informe.add_paragraph("\t0\t\t10.000\t\t" + doc_tramo1)
    informe.save(ruta)

    doc_tramo2 = str(personasTramo2) + "\t\t$" + '{:,}'.format(deb['2']).replace(',', '.')
    doc_tramo2 += "\t\t$" + '{:,}'.format(cred['2']).replace(',', '.')
    doc_tramo2 += "\t\t$" + '{:,}'.format(sald['2'][0]).replace(',', '.')
    informe.add_paragraph("10.001\t\t50.000\t\t" + doc_tramo2)
    informe.save(ruta)

    doc_tramo3 = str(personasTramo3) + "\t\t$" + '{:,}'.format(deb['3']).replace(',', '.')
    doc_tramo3 += "\t\t$" + '{:,}'.format(cred['3']).replace(',', '.')
    doc_tramo3 += "\t\t$" + '{:,}'.format(sald['3'][0]).replace(',', '.')
    informe.add_paragraph("50.001\t\t100.000\t\t" + doc_tramo3)
    informe.save(ruta)

    doc_tramo4 = str(personasTramo4) + "\t\t$" + '{:,}'.format(deb['4']).replace(',', '.')
    doc_tramo4 += "\t\t$" + '{:,}'.format(cred['4']).replace(',', '.')
    doc_tramo4 += "\t$" + '{:,}'.format(sald['4'][0]).replace(',', '.')
    informe.add_paragraph("100.001\to superior\t\t\t" + doc_tramo4)
    informe.save(ruta)

    informe.add_paragraph(
        "\t\t\t\t\t-----------------\t---------------------------------------------------------------------")
    informe.save(ruta)

    doc_totales = "Totales\t\t\t\t\t" + str(totalSocios)
    doc_totales += "\t\t$" + '{:,}'.format(totalDebitos).replace(',', '.')
    doc_totales += "\t\t$"'{:,}'.format(totalCreditos).replace(',', '.')
    doc_totales += "\t$"'{:,}'.format(saldoAcreedor).replace(',', '.')
    doc_totales += "\n\t\t\t\t\t===========\t=========================================="
    informe.add_paragraph(doc_totales)
    informe.save(ruta)

In [28]:
import docx
path_capital = "2024{}".format(nuevomes)
MES = path_capital.split("2024")[1].capitalize()
!mkdir $MES

crearDocumentoWord(capital, MES)
capital[['Nombre', 'Rut', 'Saldo', 'Cuotas']].to_excel("{}/{}.xlsx".format(MES,path_capital), index=False)